In [1]:
import numpy as np
from __future__ import division
import scipy.spatial as spatial
from vtk_rw import read_vtk
import sys
from bintrees import FastAVLTree

In [2]:
# load the meshes
complex_v,complex_f, complex_d = read_vtk('/scr/ilz3/myelinconnect/struct/surf_lh/orig/mid_surface/BP4T_lh_mid.vtk')

In [3]:
simple_v, simple_f, simple_d = read_vtk('/scr/ilz3/myelinconnect/groupavg/indv_space/BP4T/lowres_lh_d_def.vtk')

## Map simple to complex mesh

In [4]:
# find those points on the individuals complex mesh that correspond best to the simplified group mesh in subject space
inaccuracy, mapping = spatial.KDTree(complex_v).query(simple_v)

In [5]:
# find coordinates of those points in the highres mesh
voronoi_seed_idx = mapping
voronoi_seed_coord = complex_v[mapping]

In [6]:
# double check differences
dist = np.linalg.norm((voronoi_seed_coord - simple_v), axis=1)
if any(dist != inaccuracy):
    sys.exit("simple to complex mesh mapping not correct")

## Convert highres mesh into graph containing edge length

In [7]:
def graph_from_mesh(nodes, triangles, node_coords= False, edge_length=False):
    import networkx as nx
    
    # initiate empty graph
    G=nx.Graph()
    
    # add node indices
    G.add_nodes_from(range(len(nodes)))
    
    # add edges
    G.add_edges_from(triangles[:,[0,1]])
    G.add_edges_from(triangles[:,[0,2]])
    G.add_edges_from(triangles[:,[1,2]])
    
    # caution! this adds a key 'coords' to the node
    # which will also be picked up by .neighbors methods
    if node_coords:
        for n in G.nodes_iter():
            G[n]['coords']=nodes[n]
    
    if edge_length:
        for e in G.edges_iter():
            G[e[0]][e[1]]['length']=np.linalg.norm(nodes[e[0]]-nodes[e[1]])

    return G

In [8]:
complex_graph = graph_from_mesh(complex_v, complex_f, edge_length=True)

## Initiate labelling

In [49]:
# make a labelling container to be filled with the search tree
# first column are the vertex indices of the complex mesh
# second column are the labels from the simple mesh (-1 for all but the corresponding points for now)
labelling = np.zeros((complex_v.shape[0],2), dtype='int64')-1
labelling[:,0] = range(complex_v.shape[0])

for i in range(voronoi_seed_idx.shape[0]):
    labelling[voronoi_seed_idx[i]][1] = i

## Initiate AVLTree for binary search

In [50]:
# build the binary tree
# binary search trees: https://en.wikipedia.org/wiki/Binary_search_tree
# balanced binary trees: https://en.wikipedia.org/wiki/AVL_tree, 
# using them as heaps: http://samueldotj.com/blog/?s=binary+tree
# implementation used: https://pypi.python.org/pypi/bintrees/2.0.2 (cython implementation)
tree = FastAVLTree()
# organisation of the tree will be
# key: edge length
# value: tuple of vertices (source, target)

In [51]:
# function to find unlabelled neighbours of a node in the graph and add them to the tree correctly
def add_neighbours(node, graph, labels, tree):

    # find direct neighbours of the node
    neighbours = np.array(graph.neighbors(node))
    
    # check that they don't already have a label
    unlabelled = neighbours[np.where(labels[neighbours][:,1]==-1)[0]]
    
    # insert source neighbour pair with edge length to tree
    for u in unlabelled:
        tree.insert(graph[node][u]['length'],(node, u))
    
    return tree

In [52]:
# find all neighbours of the voronoi seeds
for v in voronoi_seed_idx:
    add_neighbours(v, complex_graph, labelling, tree)

## Competetive fast marching

In [ ]:
%%time
while any(labelling[:,1]==-1):
    
    while tree.count > 0:
        # pop the item with minimum edge length
        min_item = tree.pop_min()
        source = min_item[1][0]
        target = min_item[1][1]

        #if target no label yet (but source does!), assign label of source
        if labelling[target][1] == -1:
            if labelling[source][1] == -1:
                sys.exit('Source has no label, something went wrong!')
            else:
                # assign label of source to target
                labelling[target][1] = labelling[source][1]

        # add neighbours of target to tree
        add_neighbours(target, complex_graph, labelling, tree)

In [22]:
from vtk_rw import write_vtk

In [31]:
write_vtk('/scr/ilz3/myelinconnect/groupavg/indv_space/BP4T/labelling_lh.vtk', complex_v, complex_f, data=labelling[:,1, np.newaxis])